In [257]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xlrd
import os

In [336]:
base_dir = os.path.join(os.getcwd(),'..', '..', '..')
data_dir = os.path.join(base_dir, 'data/elec_price/')
gas_data = pd.read_excel(data_dir + 'gas_price_data.xls')
income_data = pd.read_csv(data_dir + 'income_data.csv', sep= ',')
elec_price_full_data = pd.read_csv(data_dir + 'PriceMT_full_table.csv')

### Income data


In [337]:
income_data.drop(columns='GeoFips',inplace = True)
income_data.iloc[1,0] = 'Alaska'
income_data.iloc[11,0] = 'Hawaii'
income_data.drop(index = 8,inplace=True)
income_data = income_data.reset_index(drop=True)

In [ ]:
expanded_income_data = pd.DataFrame()
income_data.set_index('GeoName', inplace=True)
income_data.index.name = None
for col in income_data.columns:
    period_str = col.replace(":", "")
    start_month = pd.Period(period_str, freq="Q").to_timestamp()
    months = [start_month + pd.DateOffset(months=i) for i in range(3)]
    for month in months:
        expanded_income_data[month.strftime("%Y-%m")] = income_data[col]
    
expanded_income_data.columns = pd.to_datetime(expanded_income_data.columns, format='%Y-%m')
expanded_income_data = expanded_income_data.loc[:,(expanded_income_data.columns >= '1990-01') & (expanded_income_data.columns < '2000-01')]

In [339]:
row_income_data = expanded_income_data.copy()
row_income_data.insert(0,'state',elec_price_full_data.columns.to_list())
row_income_data = row_income_data.reset_index(drop=True)
new_columns = [row_income_data.columns[0]] + pd.to_datetime(row_income_data.columns[1:]).strftime('%Y-%m-%d').tolist()
row_income_data.columns = new_columns

row_income_data_train = row_income_data.iloc[:,0:97]
row_income_data_train.to_csv(data_dir + 'row_income_data_train.txt', index=False, header = None)

expanded_income_data = expanded_income_data.T
expanded_income_data.columns = elec_price_full_data.columns.to_list()
expanded_income_data.to_csv(data_dir + 'income_data_full_table.csv', index=False, header = True)

### Gas Price data


In [310]:
gas_data = pd.read_excel(data_dir + 'gas_price_data.xls')
gas_data.set_index('Date', inplace=True)
gas_data.drop(columns='U.S. Natural Gas Citygate Price (Dollars per Thousand Cubic Feet)',inplace = True)
gas_data.index.name = None
gas_data.columns = elec_price_full_data.columns.to_list()
gas_data = gas_data.loc[(gas_data.index >= '1990-01-01') & (gas_data.index < '2000-01-01'),:]
nans = np.where(pd.isna(gas_data))
for i in range(len(np.where(pd.isna(gas_data))[0])):
    gas_data.iloc[nans[0][i],nans[1][i]] = gas_data.iloc[nans[0][i]-1,nans[1][i]]


In [315]:
gas_data_row = gas_data.transpose()
gas_data_row.insert(0,'state',gas_data_row.index)
gas_data_row = gas_data_row.reset_index(drop=True)
new_columns = [gas_data_row.columns[0]] + pd.to_datetime(gas_data_row.columns[1:]).strftime('%Y-%m-%d').tolist()
gas_data_row.columns = new_columns
gas_data_row_train = gas_data_row.iloc[:,0:97]

In [316]:
gas_data_row_train.to_csv(data_dir + 'gas_data_train.txt', index=False, header = None)
gas_data.to_csv(data_dir + 'gas_data_full_table.csv', index=False, header = True)